### This note book runs the efficent model using the noisy students weights. We will use the features and fine tune it with our dataset.
### we do not have enough data for training it from scratch

Remember whenever learning from scractch for EfficientNet
Note: each Keras Application expects a specific kind of input preprocessing. For EfficientNet, input preprocessing is included as part of the model (as a Rescaling layer), and thus tf.keras.applications.efficientnet.preprocess_input is actually a pass-through function. EfficientNet models expect their inputs to be float tensors of pixels with values in the [0-255] range.



In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np
import pathlib
import cv2
import PIL
import PIL.Image
import time
import datetime
import matplotlib.pylab as plt
import tensorboard
SEEDS = 42
np.random.seed(SEEDS)
tf.random.set_seed(SEEDS)
import itertools
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
GPU = tf.config.list_physical_devices('GPU')
CPU= tf.config.list_physical_devices('CPU')
print("Num GPUs:", len(GPU))

print("Num CPUs:", len(CPU))

### set the memory growth incremental

In [ ]:
## The logical GPUs is not supported in the tensflow available binary, you need to compile it from scratch for multiple logical GPUs

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
      tf.config.experimental.set_memory_growth(gpus[0], True)
      print(len(gpus), "Physical GPUs,")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

### defining training dataset diredtories

In [ ]:
data_dir = "/home/pms/PMS-dataset/data-set/combined/training"
data_dir = pathlib.Path(data_dir)
print("Training-Directory: ",data_dir)


image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

### preparing dataset from the directory into training and validation data

In [ ]:
IMAGE_WIDTH = 600
IMAGE_HEIGHT = 600
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
BATCH_SIZE = 1
TOTAL_CLASSES = 10
train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_dir),
  validation_split=0.2,
  subset="training",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_dir),
  validation_split=0.2,
  subset="validation",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

class_names = train_ds.class_names

plt.figure(figsize=(2, 2))
for images, labels in train_ds.take(1):
  for i in range(4):
    ax = plt.subplot(2, 2, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

class_names = np.array(train_ds.class_names)
print(class_names)
class_names_val = np.array(val_ds.class_names)
print(class_names_val)

### Apply Data augmentation to the training set and rescaling the images to 0-1 in both training and validation set and autotune

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomFlip('vertical'),
  tf.keras.layers.RandomRotation(2),
  tf.keras.layers.RandomZoom(.5),
  tf.keras.layers.RandomContrast(0.1, 0.2)
])
# apply normalization and augmentation to the training set and validation set
normalization_layer = tf.keras.layers.Rescaling(1./255)

## WARNING: the rescaling layer is not applied if uisng keras EfficientNet

#train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
#val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# check the batch size
image_batch, label_batch = next(iter(train_ds.take(1)))
print(image_batch.shape, label_batch.shape)

In [ ]:
## for debugging purpose
#tf.debugging.set_log_device_placement(True) # this line is to turn on debugging information

train_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=3, restore_best_weights=True
    )
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [ ]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
x = (inputs)
outputs = EfficientNetB7(include_top=True, weights=None, classes=TOTAL_CLASSES)(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.001
## choose the optimizer uncomment the one you want

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#              metrics=['accuracy'])

model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate, momentum=0.9), 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


model.summary()

In [ ]:
NUM_EPOCHS = 100
class_weights = {0: 0.85,1 : 0.83,2 : 1.495,3 : 1.57,4 : 2.153,5 : 2.53,6 : 1.17,7 : 0.89,8 : 0.609,9 : 0.54}

history = model.fit(
        train_ds,
        epochs=NUM_EPOCHS,
        validation_data=val_ds,
        callbacks=tensorboard_callback,
        
        class_weight=class_weights
    )

In [ ]:
def plot_hist(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,3.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()


plot_hist(history)

In [ ]:
reloaded = model
with tf.device('device:GPU:0'):
  test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/combined/testing')
  print(test_data_dir)
  img_height = 224
  img_width = 224
  batch_size = 1
  class_number = 10

  test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
  #test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

  loss, accuracy = reloaded.evaluate(test_dataset)
  print('Test accuracy :', accuracy)
  print('Test loss :', loss)

In [ ]:
with tf.device('device:GPU:0'):
    result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/combined/testing')
    output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/combined/output_test-efficientnetB7-scratch-3epochs-batch-size-1')
    try:
        os.makedirs(output_test_dir, exist_ok=True)
        print("Directory created")
    except OSError as e:
        print("Directory already exists")
count = 0
countf = 0
countF = 0
countT = 0
count_loop = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
count6 = 0
count7 = 0
count8 = 0
count9 = 0
count10 = 0

root, class_n, files = next(os.walk(result_test_data_dir))
print(class_n)
for c in class_n:
    new_root = os.path.join(output_test_dir, c)
    try:
        os.mkdir(new_root)
    except OSError as e:
        print("Directory already exists")
    for dirs in os.walk(os.path.join(root, c)):
        for files in dirs:
            for filename in files:
                if(filename.endswith(".jpg")):
                    path = os.path.join(root, c, filename)
                    #img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
                    orig = cv2.imread(path)
                    img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (img_height, img_width))
                    img_array = tf.keras.utils.img_to_array(img)
                    img_array = tf.expand_dims(img_array, 0) # Create a batch
                    #img_array = normalization_layer(img_array)
                    with tf.device('device:GPU:0'):
                        predictions = reloaded.predict(img_array)
                        score = tf.nn.softmax(predictions[0])
                        y_pred = tf.math.argmax(score)
                        y_pred = int(y_pred)
                    if (int(class_names[np.argmax(score)])==int(c)):
                        count=count+1
                    elif(int(class_names[np.argmax(score)]) ==10):
                        countf=countf+1
                        count10=count10+1
                    elif(int(class_names[np.argmax(score)])==9):
                        countf=countf+1
                        count9=count9+1
                    elif(int(class_names[np.argmax(score)]) ==8):
                        countf=countf+1
                        count8=count8+1
                    elif(int(class_names[np.argmax(score)]) ==7):
                        countf=countf+1
                        count7=count7+1
                    elif(int(class_names[np.argmax(score)]) ==6):
                        countf=countf+1
                        count6=count6+1
                    elif(int(class_names[np.argmax(score)]) ==5):
                        countf=countf+1
                        count5=count5+1
                    elif(int(class_names[np.argmax(score)]) ==4):
                        countf=countf+1
                        count4=count4+1
                    elif(int(class_names[np.argmax(score)]) ==3):
                        countf=countf+1
                        count3=count3+1
                    elif(int(class_names[np.argmax(score)])==2):
                        countf=countf+1
                        count2=count2+1
                    elif(int(class_names[np.argmax(score)])==1):
                        countf=countf+1
                        count1=count1+1
                    else:
                        print('error')
                    new_filename = filename.split(".")[0]+ filename.split(".")[1] + "_" + "P_" + str(int(class_names[np.argmax(score)])) + ".jpg"
                    new_path = os.path.join(new_root, new_filename)
                    cv2.imwrite(new_path, orig)
                    #print(new_filename )
    countT = countT + count
    countF = countF + countf
    print('=========================================================')
    print('class: ',c, 'Images: ',len(files), 'TP: ', count/len(files), 'FP: ',countf/len(files))
    #print('one : ',count1/len(files),'two: ',count2/len(files),'three: ',count3/len(files),'four: ',count4/len(files),'five: ',count5/len(files),'six: ',count6/len(files),'seven: ',count7/len(files),'eight: ',count8/len(files),'nine: ',count9/len(files),'ten: ',count10/len(files))
    print('one: ',count1,'two: ',count2,'three: ',count3,'four: ',count4,'five: ',count5,'six: ',count6,'seven: ',count7,'eight: ',count8,'nine: ',count9,'ten: ',count10)
    test.write(int(c), 1, count1)
    test.write(int(c), 2, count2)
    test.write(int(c), 3, count3)
    test.write(int(c), 4, count4)
    test.write(int(c), 5, count5)
    test.write(int(c), 6, count6)
    test.write(int(c), 7, count7)
    test.write(int(c), 8, count8)
    test.write(int(c), 9, count9)
    test.write(int(c), 10, count10)
    test.write(int(c), 11, count)
    test.write(int(c), 12, countf)

    count = 0
    countf = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    count6 = 0
    count7 = 0
    count8 = 0
    count9 = 0
    count10 = 0
    count_loop = count_loop + 1
print('Total classes: ',count_loop,'Total TP: ', countT/(countT+countF), 'Total FP: ',countF)
test.write(11, 0, countT/(countT+countF))
test.write(11, 1, countF)

In [ ]:
wb.save('test-efficientnetB7-with-weight-scratch.xls')